In [ ]:
%sh
curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -
curl https://packages.microsoft.com/config/ubuntu/16.04/prod.list > /etc/apt/sources.list.d/mssql-release.list
sudo apt-get update
sudo ACCEPT_EULA=Y apt-get -q -y install msodbcsql17

In [ ]:
import os

from pyspark.sql import SparkSession
from pyspark.dbutils import DBUtils

from edm.disaster_recovery.synapse_delta_tables_sync import SyncPolybaseDeltaTables

In [ ]:
spark = SparkSession.builder.appName('').getOrCreate()
dbutils = DBUtils(spark)

In [ ]:
dbutils.widgets.text('ingestion_type','')
dbutils.widgets.text('adls_account_name', '')
dbutils.widgets.text('kv_name', '')
dbutils.widgets.text('config_path', '')
dbutils.widgets.text('synapse_server_name', '')
dbutils.widgets.text('synapse_db', '')
dbutils.widgets.text('spn_id', '')
dbutils.widgets.text('tenant_id', '')

In [ ]:
ingestion_type = dbutils.widgets.get('ingestion_type')
spn_id = dbutils.widgets.get('spn_id')
tenant_id = dbutils.widgets.get('tenant_id')
kv_name = dbutils.widgets.get('kv_name')
adls_account_name = dbutils.widgets.get('adls_account_name')
config_path = dbutils.widgets.get('config_path')
synapse_server_name = dbutils.widgets.get('synapse_server_name')
synapse_db = dbutils.widgets.get('synapse_db')

In [ ]:
spn_password = os.getenv('SPN_PASSWORD', None)

In [ ]:
spn_credentials = dict()
spn_credentials['spn_id'] = spn_id
spn_credentials['spn_password'] = spn_password
spn_credentials['tenant_id'] = tenant_id

In [ ]:
obj = SyncPolybaseDeltaTables(
        ingestion_type, adls_account_name, spn_credentials,
        config_path, kv_name, spark, synapse_server_name, synapse_db)
obj.run()